# Constuire des agents

Nous allons construire un email assistant à partir de zéro, en commençant ici par 1) l’agent architecture (avec LangGraph), puis enchaînant avec 2) testing (avec LangSmith), 3) human-in-the-loop, et 4) memory.

Ce diagramme montre comment ces éléments vont s’imbriquer ensemble :

![overview-img](img/overview.png)

### Charger les variables d’environnement

In [1]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

## Tool Definition
Commençons par définir quelques outils simples que notre email assistant utilisera grâce au décorateur `@tool` :

In [2]:
from typing import Literal
from datetime import datetime
from pydantic import BaseModel
from langchain_core.tools import tool

@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}' and content: {content}"

@tool
def schedule_meeting(
    attendees: list[str], subject: str, duration_minutes: int, preferred_day: datetime, start_time: int
) -> str:
    """Schedule a calendar meeting."""
    # Placeholder response - in real app would check calendar and schedule
    date_str = preferred_day.strftime("%A, %B %d, %Y")
    return f"Meeting '{subject}' scheduled on {date_str} at {start_time} for {duration_minutes} minutes with {len(attendees)} attendees"

@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

@tool
class Done(BaseModel):
      """E-mail has been sent."""
      done: bool

## Building our email assistant

Nous allons combiner un router et un agent pour construire notre email assistant.

![agent_workflow_img](img/email_workflow.png)

### Router

L’étape de routing gère la décision de triage.

Le **triage router** se concentre uniquement sur la décision de triage, tandis que l’agent se concentre uniquement sur la réponse.

### State

Lors de la construction d’un agent, il est important de réfléchir aux informations que vous souhaitez suivre dans le temps. Nous utiliserons l’objet pré-construit `MessagesState` de LangGraph, qui est simplement un dictionnaire avec une clé `messages` dont la logique de mise à jour consiste à ajouter (append) les messages renvoyés par les nœuds. Toutefois, LangGraph offre la flexibilité de suivre d’autres informations. Nous définirons un `State` personnalisé qui étend `MessagesState` et ajoute une clé `classification_decision`.

In [5]:
from langgraph.graph import MessagesState

class State(MessagesState):
    # We can add a specific key to our state for the email input
    email_input: dict
    classification_decision: Literal["ignore", "respond", "notify"]

#### Triage node

Nous définissons une fonction python avec notre logique de **triage routing.**

Pour cela, nous utilisons des structured outputs avec un modèle **Pydantic**, particulièrement utile pour définir des **structured output schemas** grâce aux indications de type (type hints) et à la validation. Les descriptions dans le modèle **Pydantic** sont importantes, car elles sont transmises au LLM sous forme de JSON schema afin d’orienter la **output coercion**.

In [ ]:
%load_ext autoreload
%autoreload 2

from pydantic import BaseModel, Field
from email_assistant.utils import parse_email, format_email_markdown
from email_assistant.prompts import triage_system_prompt, triage_user_prompt, default_triage_instructions, default_background
from langchain.chat_models import init_chat_model
from langgraph.graph import END
from langgraph.types import Command

In [ ]:
from rich.markdown import Markdown
Markdown(triage_system_prompt)

In [ ]:
Markdown(triage_user_prompt)

In [ ]:
Markdown(default_background)

In [ ]:
Markdown(default_triage_instructions)

In [ ]:
class RouterSchema(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

# Initialize the LLM for use with router / structured output
llm = init_chat_model("openai:gpt-4.1", temperature=0.0)
llm_router = llm.with_structured_output(RouterSchema) 

def triage_router(state: State) -> Command[Literal["response_agent", "__end__"]]:
    """Analyze email content to decide if we should respond, notify, or ignore."""
    
    author, to, subject, email_thread = parse_email(state["email_input"])
    system_prompt = triage_system_prompt.format(
        background=default_background,
        triage_instructions=default_triage_instructions
    )

    user_prompt = triage_user_prompt.format(
        author=author, to=to, subject=subject, email_thread=email_thread
    )

    result = llm_router.invoke(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )
    
    if result.classification == "respond":
        print("📧 Classification: RESPOND - This email requires a response")
        goto = "response_agent"
        update = {
            "messages": [
                {
                    "role": "user",
                    "content": f"Respond to the email: \n\n{format_email_markdown(subject, author, to, email_thread)}",
                }
            ],
            "classification_decision": result.classification,
        }
        
    elif result.classification == "ignore":
        print("🚫 Classification: IGNORE - This email can be safely ignored")
        goto = END
        update =  {
            "classification_decision": result.classification,
        }
        
    elif result.classification == "notify":
        print("🔔 Classification: NOTIFY - This email contains important information")
        # For now, we go to END. But we will add to this later!
        goto = END
        update = {
            "classification_decision": result.classification,
        }
        
    else:
        raise ValueError(f"Invalid classification: {result.classification}")
    return Command(goto=goto, update=update)

Command objects

Nous utilisons des Command objects dans LangGraph à la fois pour mettre à jour le state et sélectionner le next node à visiter. C’est une alternative utile aux edges.

### Agent

Maintenant, construisons l’agent.

#### LLM node

Ici, nous définissons le LLM decision-making node. Ce nœud reçoit le current state, appelle le LLM, et met à jour `messages` avec la sortie du LLM.

Nous enforçons l’usage des tools avec OpenAI en réglant `tool_choice="required`".

In [ ]:
from email_assistant.tools.default.prompt_templates import AGENT_TOOLS_PROMPT
from email_assistant.prompts import agent_system_prompt, default_response_preferences, default_cal_preferences

In [ ]:
Markdown(AGENT_TOOLS_PROMPT)

In [ ]:
Markdown(agent_system_prompt)

In [ ]:
# Collect all tools
tools = [write_email, schedule_meeting, check_calendar_availability, Done]
tools_by_name = {tool.name: tool for tool in tools}

# Initialize the LLM, enforcing tool use
llm = init_chat_model("openai:gpt-4.1", temperature=0.0)
llm_with_tools = llm.bind_tools(tools, tool_choice="any")

def llm_call(state: State):
    """LLM decides whether to call a tool or not"""

    return {
        "messages": [
            # Invoke the LLM
            llm_with_tools.invoke(
                # Add the system prompt
                [   
                    {"role": "system", "content": agent_system_prompt.format(
                        tools_prompt=AGENT_TOOLS_PROMPT,
                        background=default_background,
                        response_preferences=default_response_preferences,
                        cal_preferences=default_cal_preferences, 
                    )}
                ]
                # Add the current messages to the prompt
                + state["messages"]
            )
        ]
    }

#### Tool handler node

Après que le LLM a pris une décision, nous devons exécuter le tool choisi.

Le nœud `tool_handler` exécute le tool. Nous voyons que les nodes peuvent update le graph state pour capturer toute modification d’état importante, comme la classification decision.

In [ ]:
def tool_handler(state: State):
    """Performs the tool call."""

    # List for tool messages
    result = []
    
    # Iterate through tool calls
    for tool_call in state["messages"][-1].tool_calls:
        # Get the tool
        tool = tools_by_name[tool_call["name"]]
        # Run it
        observation = tool.invoke(tool_call["args"])
        # Create a tool message
        result.append({"role": "tool", "content" : observation, "tool_call_id": tool_call["id"]})
    
    # Add it to our messages
    return {"messages": result}

#### Conditional Routing

Notre agent doit décider quand continuer à utiliser des tools et quand s’arrêter. Cette fonction de conditional routing oriente l’agent soit à continue, soit à terminate.

In [ ]:
def should_continue(state: State) -> Literal["tool_handler", "__end__"]:
    """Route to tool handler, or end if Done tool called."""
    
    # Get the last message
    messages = state["messages"]
    last_message = messages[-1]
    
    # Check if it's a Done tool call
    if last_message.tool_calls:
        for tool_call in last_message.tool_calls: 
            if tool_call["name"] == "Done":
                return END
            else:
                return "tool_handler"

#### Agent Graph

Enfin, nous pouvons assembler tous les composants.

In [ ]:
from langgraph.graph import StateGraph, START, END
from email_assistant.utils import show_graph

# Build workflow
overall_workflow = StateGraph(State)

# Add nodes
overall_workflow.add_node("llm_call", llm_call)
overall_workflow.add_node("tool_handler", tool_handler)

# Add edges
overall_workflow.add_edge(START, "llm_call")
overall_workflow.add_conditional_edges(
    "llm_call",
    should_continue,
    {
        "tool_handler": "tool_handler",
        END: END,
    },
)
overall_workflow.add_edge("tool_handler", "llm_call")

# Compile the agent
agent = overall_workflow.compile()

In [ ]:
# View
show_graph(agent)

Cela crée un graph qui :

1. Starts with an LLM decision

2. Conditionally routes vers tool execution ou termination

3. Après tool execution, revient au LLM pour la next decision

4. Repeats jusqu’à completion ou si aucun tool n’est appelé

### Combine workflow with our agent

Nous pouvons combine le router et l’agent.

In [ ]:
overall_workflow = (
    StateGraph(State)
    .add_node(triage_router)
    .add_node("response_agent", agent)
    .add_edge(START, "triage_router")
).compile()

In [ ]:
show_graph(overall_workflow, xray=True)

C’est une composition de plus haut niveau où :

1. D’abord, le triage router analyse l’email

2. Si nécessaire, le response agent se charge de rédiger une réponse

3. Le workflow se termine soit lorsque le triage décide qu’aucune réponse n’est nécessaire, soit lorsque le response agent a terminé

In [ ]:
email_input = {
    "author": "System Admin <sysadmin@company.com>",
    "to": "Development Team <dev@company.com>",
    "subject": "Scheduled maintenance - database downtime",
    "email_thread": "Hi team,\n\nThis is a reminder that we'll be performing scheduled maintenance on the production database tonight from 2AM to 4AM EST. During this time, all database services will be unavailable.\n\nPlease plan your work accordingly and ensure no critical deployments are scheduled during this window.\n\nThanks,\nSystem Admin Team"
}

# Run the agent
response = overall_workflow.invoke({"email_input": email_input})
for m in response["messages"]:
    m.pretty_print()

In [ ]:
email_input = {
  "author": "Alice Smith <alice.smith@company.com>",
  "to": "John Doe <john.doe@company.com>",
  "subject": "Quick question about API documentation",
  "email_thread": "Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice"
}

# Run the agent
response = overall_workflow.invoke({"email_input": email_input})
for m in response["messages"]:
    m.pretty_print()

## Testing with Local Deployment

Vous pouvez trouver le fichier de notre agent dans le répertoire :

* `src/email_assistant/email_assistant.py`


Vous pouvez les tester en local dans LangGraph Studio en exécutant :

```
! langgraph dev
```

Example e-mail you can test

Exemple d’email que vous pouvez tester :

In [ ]:
{
  "author": "Alice Smith <alice.smith@company.com>",
  "to": "John Doe <john.doe@company.com>",
  "subject": "Quick question about API documentation",
  "email_thread": "Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice"
}

![studio-img](img/studio.png)